In [36]:
using JuAFEM, Tensors, TimerOutputs, ProgressMeter
using KrylovMethods, IterativeSolvers

In [37]:
struct NeoHooke
    μ::Float64
    λ::Float64
end

function Ψ(C, mp::NeoHooke)
    μ = mp.μ
    λ = mp.λ
    Ic = tr(C)
    J = sqrt(det(C))
    return μ / 2 * (Ic - 3) - μ * log(J) + λ / 2 * log(J)^2
end

function constitutive_driver(C, mp::NeoHooke) 
    # to compute all derivatives in one function call 
    ∂²Ψ∂C², ∂Ψ∂C = Tensors.hessian(y -> Ψ(y, mp), C, :all)
    S = 2.0 * ∂Ψ∂C
    ∂S∂C = 4.0 * ∂²Ψ∂C²
    return S, ∂S∂C
end

function assemble_element!(ke, ge, cell, cv, fv, mp, ue)
    reinit!(cv, cell) #reinitialize the cell values, and reset output arrays
    fill!(ke, 0.0)
    fill!(ge, 0.0)
    
    b = Vec{3}((0.0,-0.5,0.0)) #body force
    t = Vec{3}((0.1,0.0,0.0))
    ndofs = getnbasefunctions(cv)
    
    for qp in 1:getnquadpoints(cv)
      dΩ = getdetJdV(cv,qp) #known as dΓ in my fem notation
      #computing the deformation gradient and right-cauchy green tensor C 
      ∇u = function_gradient(cv, qp, ue)
       F = one(∇u) + ∇u
       C = tdot(F)
       # Compute stress and tangent
       S, ∂S∂C = constitutive_driver(C, mp)
       P = F ⋅ S
       I = one(S)
       ∂P∂F = otimesu(F, I) ⊡ ∂S∂C ⊡ otimesu(F', I) + otimesu(I, S) #ask MAX
      
      #loop over test functions 
      for i in 1:ndofs
          #test function + gradient 
          δui = shape_value(cv,qp,i)
          ∇δui = shape_gradient(cv,qp,i)
          #add contribution to the residual from this test function 
          ge[i] += ( ∇δui ⊡ P - δui ⋅ b ) * dΩ
          
          ∇δui∂P∂F = ∇δui ⊡ ∂P∂F # Hoisted computation #ask MAX
          for j in 1:ndofs
                ∇δuj = shape_gradient(cv, qp, j)
                #add contribution to the tangent 
                ke[i,j] += ( ∇δui∂P∂F ⊡ ∇δuj ) * dΩ #ask MAX
          end 
      end
    end

    #surface integral for the traction 
    for face in 1:nfaces(cell)
        if onboundary(cell, face)
            reinit!(fv, cell, face)
            for q_point in 1:getnquadpoints(fv)
              dΓ = getdetJdV(fv, q_point)
              for i in 1:ndofs
                  δui = shape_value(fv, q_point, i)
                  ge[i] -= (δui ⋅ t)*dΓ
              end
            end
        end
    end
end 

assemble_element! (generic function with 1 method)

In [38]:
function assemble_global!(K,f, dh, cv, fv, mp, u)
    n = ndofs_per_cell(dh)
    ke = zeros(n,n)
    ge = zeros(n)
    
    #start _assemble resets K and f 
    assembler = start_assemble(K,f)
    
    #loop over all cells in the grid 
    @timeit "assemble" for cell in CellIterator(dh)
        global_dofs = celldofs(cell)
        ue = u[global_dofs] #element dofs 
        @timeit "element assemble" assemble_element!(ke,ge,cell, cv, fv, mp, ue)
        assemble!(assembler, global_dofs,ge, ke)
    end
end

assemble_global! (generic function with 1 method)

In [39]:
function solve()
    reset_timer!()
    
    #generate a grid
    N = 10
    L = 1.0 
    left = zero(Vec{3})
    right = L * ones(Vec{3})
    grid = generate_grid(Tetrahedron, (N,N,N), left, right)
    
    #Material parameters 
    E = 10.0
    v = 0.3 
    μ = E / (2(1 + v))
    λ = (E * v) / ((1 + v) * (1 - 2v))
    mp = NeoHooke(μ, λ)
    
    #finite element base 
    ip = Lagrange{3, RefTetrahedron, 1}() #lagrange shape functions of dimension 3 and 1st oder 
    qr = QuadratureRule{3, RefTetrahedron}(1) #The gauss points of 3D element of order 1 
    qr_face = QuadratureRule{2, RefTetrahedron}(1) #the gauss points of the faces which is 2D and of order 1 
    cv = CellVectorValues(qr, ip) #we combine the shape functions and the gauss points to get an object 
    fv = FaceVectorValues(qr_face, ip)
    
    #dof handler 
    dh = DofHandler(grid)
    push!(dh, :u, 3)
    close!(dh)
    
    function rotation(X, t, θ = deg2rad(60.0))
        x, y, z = X
        return t * Vec{3}(
            (0.0,
            L/2 - y + (y-L/2)*cos(θ) - (z-L/2)*sin(θ),
            L/2 - z + (y-L/2)*sin(θ) + (z-L/2)*cos(θ)
            ))
    end
    
    dbcs = ConstraintHandler(dh)
    dbc = Dirichlet(:u, getfaceset(grid, "right"), (x,t) -> [0.0, 0.0, 0.0], [1, 2, 3])
    add!(dbcs, dbc)
    dbc = Dirichlet(:u, getfaceset(grid, "left"), (x,t) -> rotation(x,t) , [1, 2, 3])
    add!(dbcs, dbc)
    close!(dbcs)
    t = 0.5
    JuAFEM.update!(dbcs, t)
    
    # pre allocotion of vectors for the solution of Newton increments 
    _ndofs = ndofs(dh)
    un = zeros(_ndofs)
    u = zeros(_ndofs)
    Δu = zeros(_ndofs)
    ΔΔu = zeros(_ndofs)
    apply!(un, dbcs)
    
    #create sparse matrix and residual vector 
    K = create_sparsity_pattern(dh)
    g = zeros(_ndofs)
    
    #perform Newton Iterations 
    newton_itr = -1
    NEWTON_TOL = 1e-8
    prog = ProgressMeter.ProgressThresh(NEWTON_TOL, "solving:")
    
    while true; newton_itr += 1
        u .= un .+ Δu #current guess 
        assemble_global!(K, g, dh, cv, fv, mp, u)
        normg = norm(g[JuAFEM.free_dofs(dbcs)])
        apply_zero!(K, g, dbcs)
        ProgressMeter.update!(prog, normg; showvalues = [(:iter, newton_itr)])

        if normg < NEWTON_TOL
            break
        elseif newton_itr > 30
            error("Reached maximum Newton iterations, aborting")
        end

        # Compute increment using cg! from IterativeSolvers.jl
        @timeit "linear solve (KrylovMethods.cg)" ΔΔu′, flag, relres, iter, resvec = KrylovMethods.cg(K, g; maxIter = 1000)
        @assert flag == 0
        @timeit "linear solve (IterativeSolvers.cg!)" IterativeSolvers.cg!(ΔΔu, K, g; maxiter=1000)

        apply_zero!(ΔΔu, dbcs)
        Δu .-= ΔΔu
    end

    # Save the solution
    @timeit "export" begin
        vtk_grid("hyperelasticity", dh) do vtkfile
            vtk_point_data(vtkfile, dh, u)
        end
    end

    print_timer(title = "Analysis with $(getncells(grid)) elements", linechars = :ascii)
    return u
end

solve (generic function with 1 method)

In [40]:
u = solve()

solving: (thresh = 1e-08, value = 0.013799)
4m  iter:  2
solving: (thresh = 1e-08, value = 1.92988e-07)
4m  iter:  4
solving: Time: 0:00:00 (6 iterations)
  iter:  5


 ------------------------------------------------------------------------------
  Analysis with 6000 elements          Time                   Allocations      
                               ----------------------   -----------------------
       Tot / % measured:            573ms / 56.0%           79.6MiB / 16.9%    

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ------------------------------------------------------------------------------
 linear solve (Iter...      5    129ms  40.3%  25.8ms    474KiB  3.44%  94.8KiB
 assemble                   6    128ms  39.9%  21.4ms   8.52MiB  63.4%  1.42MiB
   element assemble     36.0k   82.4ms  25.7%  2.29μs     0.00B  0.00%    0.00B
 linear solve (Kryl...      5   35.0ms  10.9%  7.00ms    669KiB  4.87%   134KiB
 export                     1   28.4ms  8.85%  28.4ms   3.80MiB  28.3%  3.80MiB
 ------------------------------------------------------------------------------

3993-element Array{Float64,1}:
  0.0
  0.3415063509461096
 -0.09150635094610968
  2.6971532448498215e-5
  0.2958396776436137
 -0.12618014268650996
  0.0
  0.3165063509461096
 -0.04820508075688776
  0.0
  0.2732050807568877
 -0.07320508075688775
  0.0
  ⋮
  0.004391706946162339
 -0.09132775507048106
  0.06642349477420981
  0.0018552809500001078
 -0.05748898044581073
  0.04292109221065066
 -0.0001710346041245002
 -0.02554728643553888
  0.01906350760975909
  0.0
  0.0
  0.0